In [146]:
import requests
import pandas as pd
import random
from datetime import datetime, timedelta
import streamlit as st

In [156]:
data = pd.read_csv('final-crime-data.csv')
data

,Unnamed: 0,unique_case_id,type_of_crime,location,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,...,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,timestamp,date
0,0,000001_1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße","Female, Male",Male,3,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02:15:00,2022-14-01
1,1,000002_1167410,Homicide,Kühnemannstraße,Male,Male,1,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06:00:00,2022-02-02
2,2,000003_1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,Male,Male,1,3,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01:15:00,2022-14-01
3,3,000004_1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",Male,Male,1,2,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2022-20-06
4,4,000005_1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",Unknown,Male,Unknown,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02:40:00,2022-13-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,2400,000007_1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,Unknow,Male,Unknow,1,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2023-23-05
2297,2401,000008_1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",Male and Female,Male,7,1,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18:30:00,2023-23-05
2298,2402,000009_1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,Male,Male,2,1,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12:30:00,2023-23-05
2299,2403,000010_1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",Male and Female,Unknow,40,Unknow,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13:00:00,2023-23-05


In [157]:
def coordinates(location):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': location,
        'format': 'json'
    }
    response = requests.get(url, params=params).json()
    return response
    #return (response[0]['lat'], response[0]['lon'])

In [ ]:
import pandas as pd
import csv


def process_csv(file_path, chunk_size=300):
    df_list = []
    locations = []
    with open(file_path, 'r') as csv_file:
        reader = csv.reader(csv_file)
        header = next(reader)  # Store the header row
        output_index = len(header)  # Determine the index for the "Output" column
        
        # Add "Output" to the header
        header.append("Output")
        
        for row in reader:
            location = row[0]  # Assuming the location is in the first column of the CSV
            locations.append(location)
            
            if len(locations) >= chunk_size:
                df = pd.DataFrame(locations, columns=['location'])
                df['Output'] = df['location'].apply(coordinates)
                df_list.append(df)
                locations = []  # Reset the list
        
        if len(locations) > 0:  # Process remaining locations if any
            df = pd.DataFrame(locations, columns=['location'])
            df['Output'] = df['location'].apply(coordinates)
            df_list.append(df)
        
        output_df = pd.concat(df_list)
        output_df = output_df.reset_index(drop=True)  # Reset the index
        
        # Concatenate the original DataFrame with the output DataFrame
        merged_df = pd.read_csv(file_path)
        merged_df = pd.concat([merged_df, output_df['Output']], axis=1)
        
        return merged_df

# Example usage
csv_file_path = 'final-crime-data.csv'

output_df = process_csv(csv_file_path)



In [164]:
data2 = pd.DataFrame(output_df)
data2

,Unnamed: 0,unique_case_id,type_of_crime,location,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,...,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,timestamp,date,Output
0,0,000001_1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße","Female, Male",Male,3,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02:15:00,2022-14-01,[]
1,1,000002_1167410,Homicide,Kühnemannstraße,Male,Male,1,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06:00:00,2022-02-02,"[{'place_id': 309633045, 'licence': 'Data © Op..."
2,2,000003_1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,Male,Male,1,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01:15:00,2022-14-01,"[{'place_id': 309196061, 'licence': 'Data © Op..."
3,3,000004_1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",Male,Male,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2022-20-06,"[{'place_id': 307971833, 'licence': 'Data © Op..."
4,4,000005_1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",Unknown,Male,Unknown,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02:40:00,2022-13-01,"[{'place_id': 270733258, 'licence': 'Data © Op..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,2400,000007_1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,Unknow,Male,Unknow,1,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2023-23-05,"[{'place_id': 308716962, 'licence': 'Data © Op..."
2297,2401,000008_1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",Male and Female,Male,7,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18:30:00,2023-23-05,"[{'place_id': 307946142, 'licence': 'Data © Op..."
2298,2402,000009_1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,Male,Male,2,1,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12:30:00,2023-23-05,"[{'place_id': 308481959, 'licence': 'Data © Op..."
2299,2403,000010_1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",Male and Female,Unknow,40,Unknow,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13:00:00,2023-23-05,"[{'place_id': 307984029, 'licence': 'Data © Op..."


In [166]:
data3=data2.reset_index().drop(columns=['index','Unnamed: 0'])
data3['Good_output'] = data3['Output'].apply(lambda x : x[0] if x!=[] else x
                                             

,unique_case_id,type_of_crime,location,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,Hate Crime - Gender,...,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,timestamp,date,Output,Good_output
0,000001_1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße","Female, Male",Male,3,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,02:15:00,2022-14-01,[],[]
1,000002_1167410,Homicide,Kühnemannstraße,Male,Male,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,06:00:00,2022-02-02,"[{'place_id': 309633045, 'licence': 'Data © Op...","{'place_id': 309633045, 'licence': 'Data © Ope..."
2,000003_1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,Male,Male,1,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,01:15:00,2022-14-01,"[{'place_id': 309196061, 'licence': 'Data © Op...","{'place_id': 309196061, 'licence': 'Data © Ope..."
3,000004_1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",Male,Male,1,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2022-20-06,"[{'place_id': 307971833, 'licence': 'Data © Op...","{'place_id': 307971833, 'licence': 'Data © Ope..."
4,000005_1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",Unknown,Male,Unknown,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,02:40:00,2022-13-01,"[{'place_id': 270733258, 'licence': 'Data © Op...","{'place_id': 270733258, 'licence': 'Data © Ope..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,000007_1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,Unknow,Male,Unknow,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2023-23-05,"[{'place_id': 308716962, 'licence': 'Data © Op...","{'place_id': 308716962, 'licence': 'Data © Ope..."
2297,000008_1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",Male and Female,Male,7,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18:30:00,2023-23-05,"[{'place_id': 307946142, 'licence': 'Data © Op...","{'place_id': 307946142, 'licence': 'Data © Ope..."
2298,000009_1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,Male,Male,2,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,12:30:00,2023-23-05,"[{'place_id': 308481959, 'licence': 'Data © Op...","{'place_id': 308481959, 'licence': 'Data © Ope..."
2299,000010_1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",Male and Female,Unknow,40,Unknow,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,13:00:00,2023-23-05,"[{'place_id': 307984029, 'licence': 'Data © Op...","{'place_id': 307984029, 'licence': 'Data © Ope..."


In [167]:
data_empty = data3[data3['Good_output'].map(lambda x: len(x)) == 0]
data_empty

,unique_case_id,type_of_crime,location,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,Hate Crime - Gender,...,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,timestamp,date,Output,Good_output
0,000001_1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße","Female, Male",Male,3,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,02:15:00,2022-14-01,[],[]


In [168]:
data3['lon'] = data3['Good_output'].apply(lambda x : x['lon'] if x!=[] else x)
data3['lat'] = data3['Good_output'].apply(lambda x : x['lat'] if x!=[] else x)
data3

,unique_case_id,type_of_crime,location,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,Hate Crime - Gender,...,Domestic Violence,Missing Person,Traffic Incident,Unclassified,timestamp,date,Output,Good_output,lon,lat
0,000001_1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße","Female, Male",Male,3,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,02:15:00,2022-14-01,[],[],[],[]
1,000002_1167410,Homicide,Kühnemannstraße,Male,Male,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,06:00:00,2022-02-02,"[{'place_id': 309633045, 'licence': 'Data © Op...","{'place_id': 309633045, 'licence': 'Data © Ope...",30.982038361503804,-6.4107954
2,000003_1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,Male,Male,1,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,01:15:00,2022-14-01,"[{'place_id': 309196061, 'licence': 'Data © Op...","{'place_id': 309196061, 'licence': 'Data © Ope...",36.90436132877197,-3.3253383000000003
3,000004_1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",Male,Male,1,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,2022-20-06,"[{'place_id': 307971833, 'licence': 'Data © Op...","{'place_id': 307971833, 'licence': 'Data © Ope...",-91.153628,39.548248
4,000005_1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",Unknown,Male,Unknown,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,02:40:00,2022-13-01,"[{'place_id': 270733258, 'licence': 'Data © Op...","{'place_id': 270733258, 'licence': 'Data © Ope...",26.5293451,67.0655921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,000007_1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,Unknow,Male,Unknow,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,2023-23-05,"[{'place_id': 308716962, 'licence': 'Data © Op...","{'place_id': 308716962, 'licence': 'Data © Ope...",18.92547927252444,46.9457587
2297,000008_1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",Male and Female,Male,7,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18:30:00,2023-23-05,"[{'place_id': 307946142, 'licence': 'Data © Op...","{'place_id': 307946142, 'licence': 'Data © Ope...",19.416667,63.666667
2298,000009_1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,Male,Male,2,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12:30:00,2023-23-05,"[{'place_id': 308481959, 'licence': 'Data © Op...","{'place_id': 308481959, 'licence': 'Data © Ope...",7.5132917,36.6111841
2299,000010_1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",Male and Female,Unknow,40,Unknow,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13:00:00,2023-23-05,"[{'place_id': 307984029, 'licence': 'Data © Op...","{'place_id': 307984029, 'licence': 'Data © Ope...",19,63.966667


In [215]:
data4 = data3[data3['Good_output'].map(lambda x: len(x)) > 0]
data4['official_case_id'] = data4['unique_case_id'].str[-7:].astype(int)
data4

/var/folders/y9/rtty5pb54zqgkmpnmlg0gxqw0000gn/T/ipykernel_97714/387648920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['official_case_id'] = data4['unique_case_id'].str[-7:].astype(int)


,unique_case_id,type_of_crime,location,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,Hate Crime - Gender,...,Missing Person,Traffic Incident,Unclassified,timestamp,date,Output,Good_output,lon,lat,official_case_id
1,000002_1167410,Homicide,Kühnemannstraße,Male,Male,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,06:00:00,2022-02-02,"[{'place_id': 309633045, 'licence': 'Data © Op...","{'place_id': 309633045, 'licence': 'Data © Ope...",30.982038361503804,-6.4107954,1167410
2,000003_1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,Male,Male,1,3,0.0,0.0,0.0,...,0.0,0.0,0.0,01:15:00,2022-14-01,"[{'place_id': 309196061, 'licence': 'Data © Op...","{'place_id': 309196061, 'licence': 'Data © Ope...",36.90436132877197,-3.3253383000000003,1167332
3,000004_1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",Male,Male,1,2,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,2022-20-06,"[{'place_id': 307971833, 'licence': 'Data © Op...","{'place_id': 307971833, 'licence': 'Data © Ope...",-91.153628,39.548248,1167270
4,000005_1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",Unknown,Male,Unknown,1,0.0,0.0,0.0,...,0.0,0.0,0.0,02:40:00,2022-13-01,"[{'place_id': 270733258, 'licence': 'Data © Op...","{'place_id': 270733258, 'licence': 'Data © Ope...",26.5293451,67.0655921,1167075
5,000001_1318967,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",Male,Male,1,4,0.0,0.0,0.0,...,0.0,0.0,0.0,13:40:00,2023-29-04,"[{'place_id': 309355667, 'licence': 'Data © Op...","{'place_id': 309355667, 'licence': 'Data © Ope...",33.07227630402895,-6.08971685,1318967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,000007_1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,Unknow,Male,Unknow,1,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,2023-23-05,"[{'place_id': 308716962, 'licence': 'Data © Op...","{'place_id': 308716962, 'licence': 'Data © Ope...",18.92547927252444,46.9457587,1327301
2297,000008_1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",Male and Female,Male,7,1,0.0,0.0,0.0,...,0.0,0.0,0.0,18:30:00,2023-23-05,"[{'place_id': 307946142, 'licence': 'Data © Op...","{'place_id': 307946142, 'licence': 'Data © Ope...",19.416667,63.666667,1327284
2298,000009_1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,Male,Male,2,1,0.0,0.0,0.0,...,0.0,0.0,0.0,12:30:00,2023-23-05,"[{'place_id': 308481959, 'licence': 'Data © Op...","{'place_id': 308481959, 'licence': 'Data © Ope...",7.5132917,36.6111841,1327174
2299,000010_1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",Male and Female,Unknow,40,Unknow,0.0,0.0,0.0,...,0.0,0.0,0.0,13:00:00,2023-23-05,"[{'place_id': 307984029, 'licence': 'Data © Op...","{'place_id': 307984029, 'licence': 'Data © Ope...",19,63.966667,1327172


In [193]:
final_data = data4.to_csv('final_table.csv')
data4

,unique_case_id,type_of_crime,location,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,Hate Crime - Gender,...,Domestic Violence,Missing Person,Traffic Incident,Unclassified,timestamp,date,Output,Good_output,lon,lat
1,000002_1167410,Homicide,Kühnemannstraße,Male,Male,1,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,06:00:00,2022-02-02,"[{'place_id': 309633045, 'licence': 'Data © Op...","{'place_id': 309633045, 'licence': 'Data © Ope...",30.982038361503804,-6.4107954
2,000003_1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,Male,Male,1,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,01:15:00,2022-14-01,"[{'place_id': 309196061, 'licence': 'Data © Op...","{'place_id': 309196061, 'licence': 'Data © Ope...",36.90436132877197,-3.3253383000000003
3,000004_1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",Male,Male,1,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,2022-20-06,"[{'place_id': 307971833, 'licence': 'Data © Op...","{'place_id': 307971833, 'licence': 'Data © Ope...",-91.153628,39.548248
4,000005_1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",Unknown,Male,Unknown,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,02:40:00,2022-13-01,"[{'place_id': 270733258, 'licence': 'Data © Op...","{'place_id': 270733258, 'licence': 'Data © Ope...",26.5293451,67.0655921
5,000001_1318967,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",Male,Male,1,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13:40:00,2023-29-04,"[{'place_id': 309355667, 'licence': 'Data © Op...","{'place_id': 309355667, 'licence': 'Data © Ope...",33.07227630402895,-6.08971685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,000007_1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,Unknow,Male,Unknow,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,2023-23-05,"[{'place_id': 308716962, 'licence': 'Data © Op...","{'place_id': 308716962, 'licence': 'Data © Ope...",18.92547927252444,46.9457587
2297,000008_1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",Male and Female,Male,7,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18:30:00,2023-23-05,"[{'place_id': 307946142, 'licence': 'Data © Op...","{'place_id': 307946142, 'licence': 'Data © Ope...",19.416667,63.666667
2298,000009_1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,Male,Male,2,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12:30:00,2023-23-05,"[{'place_id': 308481959, 'licence': 'Data © Op...","{'place_id': 308481959, 'licence': 'Data © Ope...",7.5132917,36.6111841
2299,000010_1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",Male and Female,Unknow,40,Unknow,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13:00:00,2023-23-05,"[{'place_id': 307984029, 'licence': 'Data © Op...","{'place_id': 307984029, 'licence': 'Data © Ope...",19,63.966667


In [214]:
final = pd.read_csv('final.csv')
final.dtypes

Unnamed: 0                         int64
official_case_id                   int64
type_of_crime                     object
location                          object
datetime                          object
year_date                         object
time                              object
victim_sex                        object
offender_sex                      object
number_of_victims                 object
number_of_offenders               object
homicide                         float64
hate_crime-disability            float64
hate_crime-gender                float64
hate_crime-gender_identity       float64
hate_crime-religious             float64
hate_crime-sexual_orientation    float64
hate_crime-racial/ethnicity        int64
verbal_abuse/verbal_assault      float64
property_damage                  float64
drug_offenses                    float64
general_assault                  float64
sexual_assault                   float64
sexual_harassment                float64
property_crimes 

In [210]:
final_data

In [235]:
result = pd.merge(final,data4[['official_case_id','lon', 'lat']],on='official_case_id', how='outer')
result.dtypes


Unnamed: 0                         int64
official_case_id                   int64
type_of_crime                     object
location                          object
datetime                          object
year_date                         object
time                              object
victim_sex                        object
offender_sex                      object
number_of_victims                 object
number_of_offenders               object
homicide                         float64
hate_crime-disability            float64
hate_crime-gender                float64
hate_crime-gender_identity       float64
hate_crime-religious             float64
hate_crime-sexual_orientation    float64
hate_crime-racial/ethnicity        int64
verbal_abuse/verbal_assault      float64
property_damage                  float64
drug_offenses                    float64
general_assault                  float64
sexual_assault                   float64
sexual_harassment                float64
property_crimes 

In [226]:
result=result.reset_index().drop(columns=['Unnamed: 0'])
result

KeyError: "['Unnamed: 0'] not found in axis"

In [228]:
result.to_csv('berlin_final_table.csv')

In [ ]:
data
filtered_data['color'] = 'default color'

In [178]:
data4['date']

1       2022-02-02
2       2022-14-01
3       2022-20-06
4       2022-13-01
5       2023-29-04
           ...    
2296    2023-23-05
2297    2023-23-05
2298    2023-23-05
2299    2023-23-05
2300    2023-19-04
Name: date, Length: 2300, dtype: object

In [177]:
data4['date'] = pd.to_datetime(data4['date'], format="%Y-%d-%m")
print(data4['date'])

ValueError: unconverted data remains when parsing with format "%Y-%d-%m": "-2023", at position 26. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [30]:
df3 = pd.concat([df1,df2], axis=1, join='outer', ignore_index=False)
df3 = pd.DataFrame.dropna(df3)
df3.head()

,Unnamed: 0,Time,Date,Year,Month,Day,unique_case_id,official_case_id,type_of_crime,location,...,Property Damage,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified
0,0,20:44:00,2023-05-30 20:10:14.182771,2023,5,30,000001_1318967,1318967.0,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,13:15:00,2023-04-29 20:10:14.182771,2023,4,29,000002_1318965,1318965.0,"Property Damage, General Assault",Seydelstraße bis zur Alte-Jakob-Straße,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,11:38:00,2023-03-29 20:10:14.182771,2023,3,29,000003_1317835,1317835.0,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,00:47:00,2023-05-02 20:10:14.182771,2023,5,2,000004_1317465,1317465.0,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,01:12:00,2022-11-27 20:10:14.182771,2022,11,27,000005_1317344,1317344.0,Property Damage,"Wedding, Togostraße",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
data = pd.read_csv('dummy_table.csv')
#selected_crime_type = ['Hate Crime - Disability','Hate Crime - Gender','Hate Crime - Gender Identity']
selected_date = '2023-05-30'
selected_crime_type = ['I wanna see All']

data['Date']

0      2023-05-30 20:10:14.182771
1      2023-03-29 20:10:14.182771
2      2023-05-02 20:10:14.182771
3      2022-11-27 20:10:14.182771
4      2022-07-06 20:10:14.182771
                  ...            
224    2022-10-08 20:10:14.182771
225    2023-02-05 20:10:14.182771
226    2022-10-24 20:10:14.182771
227    2022-09-14 20:10:14.182771
228    2022-07-07 20:10:14.182771
Name: Date, Length: 229, dtype: object

In [182]:
if 'I wanna see All' in selected_crime_type:
    crime_columns = [col for col in data.columns if col not in ['date', 'lat', 'lon','type_of_crime','location','unique_case_id','victim_sex','offender_sex','number_of_victims','number_of_offenders','Output','Good_output']]
    filtered_data = data[data['Date'] == str(selected_date)].groupby(['lat', 'lon', 'Date'])[crime_columns].sum().reset_index()
else:
    if len(selected_crime_type) == 0:
        filtered_data = pd.DataFrame(columns=['Date', 'lat', 'lon'])
    else:
        columns_to_select = ['Date', 'lat', 'lon'] + selected_crime_type
        #filtered_data = data.loc[:, columns_to_select]
        filtered_data = data[data['Date'] == str(selected_date)].groupby(['lat', 'lon', 'Date'])[columns_to_select].sum().reset_index()

ValueError: cannot insert Date, already exists

In [59]:
df3[df3['location'].str.contains('bis',case=False)].index
df4 = df3.drop(index=df3[df3['location'].str.contains('bis',case=False)].index)
df4

,Unnamed: 0,Time,Date,Year,Month,Day,unique_case_id,official_case_id,type_of_crime,location,...,Property Damage,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified
0,0,20:44:00,2023-05-30 20:10:14.182771,2023,5,30,000001_1318967,1318967.0,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,11:38:00,2023-03-29 20:10:14.182771,2023,3,29,000003_1317835,1317835.0,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,00:47:00,2023-05-02 20:10:14.182771,2023,5,2,000004_1317465,1317465.0,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,01:12:00,2022-11-27 20:10:14.182771,2022,11,27,000005_1317344,1317344.0,Property Damage,"Wedding, Togostraße",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,05:35:00,2022-07-06 20:10:14.182771,2022,7,6,000006_1317325,1317325.0,Attempted Suicide,Tempelhof-Schöneberg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,291,09:18:00,2022-10-08 20:10:14.182771,2022,10,8,000007_1327301,1327301.0,"Sexual Assault, Verbal Abuse",Lichtenberg,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
292,292,07:24:00,2023-02-05 20:10:14.182771,2023,2,5,000008_1327284,1327284.0,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,293,14:19:00,2022-10-24 20:10:14.182771,2022,10,24,000009_1327174,1327174.0,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
294,294,22:13:00,2022-09-14 20:10:14.182771,2022,9,14,000010_1327172,1327172.0,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
#apply geocoding 'coordinate'
df4['Output'] = df4['location'].apply(coordinates)
df4

,Unnamed: 0,Time,Date,Year,Month,Day,unique_case_id,official_case_id,type_of_crime,location,...,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Output
0,0,20:44:00,2023-05-30 20:10:14.182771,2023,5,30,000001_1318967,1318967.0,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121562969, 'licence': 'Data © Op..."
2,2,11:38:00,2023-03-29 20:10:14.182771,2023,3,29,000003_1317835,1317835.0,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 103312933, 'licence': 'Data © Op..."
3,3,00:47:00,2023-05-02 20:10:14.182771,2023,5,2,000004_1317465,1317465.0,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121847673, 'licence': 'Data © Op..."
4,4,01:12:00,2022-11-27 20:10:14.182771,2022,11,27,000005_1317344,1317344.0,Property Damage,"Wedding, Togostraße",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 232223922, 'licence': 'Data © Op..."
5,5,05:35:00,2022-07-06 20:10:14.182771,2022,7,6,000006_1317325,1317325.0,Attempted Suicide,Tempelhof-Schöneberg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 307886045, 'licence': 'Data © Op..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,291,09:18:00,2022-10-08 20:10:14.182771,2022,10,8,000007_1327301,1327301.0,"Sexual Assault, Verbal Abuse",Lichtenberg,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 308676399, 'licence': 'Data © Op..."
292,292,07:24:00,2023-02-05 20:10:14.182771,2023,2,5,000008_1327284,1327284.0,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 113408058, 'licence': 'Data © Op..."
293,293,14:19:00,2022-10-24 20:10:14.182771,2022,10,24,000009_1327174,1327174.0,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"[{'place_id': 307526888, 'licence': 'Data © Op..."
294,294,22:13:00,2022-09-14 20:10:14.182771,2022,9,14,000010_1327172,1327172.0,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 236190135, 'licence': 'Data © Op..."


In [118]:
df5=df4.reset_index().drop(columns=['index','Unnamed: 0'])
df5['Good_output'] = df5['Output'].apply(lambda x : x[0] if x!=[] else x)
#df5 = df5[df5['Good_output'].map(lambda x: len(x)) > 0]
#df5

In [121]:

df6 = df5[df5['Good_output'].map(lambda x: len(x)) == 0]
df6

,Time,Date,Year,Month,Day,unique_case_id,official_case_id,type_of_crime,location,year,...,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Output,Good_output
7,20:06:00,2023-01-13 20:10:14.182771,2023,1,13,000009_1316949,1316949.0,"Property Damage, Verbal Abuse, General Assault",Berlinweit,2023.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
8,12:47:00,2023-03-04 20:10:14.182771,2023,3,4,000010_1316885,1316885.0,Unclassified,Berlinweit,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,[],[]
11,09:34:00,2022-12-07 20:10:14.182771,2022,12,7,000003_1312133,1312133.0,"General Assault, Property Damage",U-Bahnhof Rathaus Spandau,2023.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
12,06:41:00,2023-04-10 20:10:14.182771,2023,4,10,000004_1312099,1312099.0,Drug Offenses,"U-Bahnhof Rathaus Spandau, U-Bahnhof Haselhorst",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
21,09:01:00,2023-02-17 20:10:14.182771,2023,2,17,000009_1307433,1307433.0,"Hate Crime - Racial/Ethnicity, General Assault...",S-Bahnhof Jungfernheide,2023.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,16:39:00,2022-11-08 20:10:14.182771,2022,11,8,000009_1307433,1307433.0,"Hate Crime - Racial/Ethnicity, General Assault...",S-Bahnhof Jungfernheide,2023.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
259,10:27:00,2022-12-07 20:10:14.182771,2022,12,7,000011_1307179,1307179.0,"General Assault, Verbal Abuse, Property Damage","Harry-Breslau-Park, Steglitz-Zehlendorf",2023.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
268,08:39:00,2022-09-11 20:10:14.182771,2022,9,11,000020_1306232,1306232.0,"General Assault, Property Damage, Verbal Abuse",Tram-Haltestelle Welsestr. / Falkenberger Chau...,2023.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]
278,06:39:00,2022-12-22 20:10:14.182771,2022,12,22,000030_1304657,1304657.0,Human Trafficking,Berlin/Halle(Saale)/Hof,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[]


In [122]:
df6.to_csv('emptylist_table.csv')

In [123]:
df5['lon'] = df5['Good_output'].apply(lambda x : x['lon'] if x!=[] else x)
df5['lat'] = df5['Good_output'].apply(lambda x : x['lat'] if x!=[] else x)

In [126]:
df5 = df5[df5['Good_output'].map(lambda x: len(x)) > 0]
df5

,Time,Date,Year,Month,Day,unique_case_id,official_case_id,type_of_crime,location,year,...,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Output,Good_output,lon,lat
0,20:44:00,2023-05-30 20:10:14.182771,2023,5,30,000001_1318967,1318967.0,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121562969, 'licence': 'Data © Op...","{'place_id': 121562969, 'licence': 'Data © Ope...",13.29000220854009,52.50994745
1,11:38:00,2023-03-29 20:10:14.182771,2023,3,29,000003_1317835,1317835.0,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 103312933, 'licence': 'Data © Op...","{'place_id': 103312933, 'licence': 'Data © Ope...",13.1706652,52.5192835
2,00:47:00,2023-05-02 20:10:14.182771,2023,5,2,000004_1317465,1317465.0,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121847673, 'licence': 'Data © Op...","{'place_id': 121847673, 'licence': 'Data © Ope...",13.513130233356152,52.51865085
3,01:12:00,2022-11-27 20:10:14.182771,2022,11,27,000005_1317344,1317344.0,Property Damage,"Wedding, Togostraße",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 232223922, 'licence': 'Data © Op...","{'place_id': 232223922, 'licence': 'Data © Ope...",13.3375332,52.5548933
4,05:35:00,2022-07-06 20:10:14.182771,2022,7,6,000006_1317325,1317325.0,Attempted Suicide,Tempelhof-Schöneberg,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 307886045, 'licence': 'Data © Op...","{'place_id': 307886045, 'licence': 'Data © Ope...",13.388288723500661,52.4404422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,09:18:00,2022-10-08 20:10:14.182771,2022,10,8,000007_1327301,1327301.0,"Sexual Assault, Verbal Abuse",Lichtenberg,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 308676399, 'licence': 'Data © Op...","{'place_id': 308676399, 'licence': 'Data © Ope...",7.4812268,48.9212959
291,07:24:00,2023-02-05 20:10:14.182771,2023,2,5,000008_1327284,1327284.0,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 113408058, 'licence': 'Data © Op...","{'place_id': 113408058, 'licence': 'Data © Ope...",13.364231910001136,52.54411815
292,14:19:00,2022-10-24 20:10:14.182771,2022,10,24,000009_1327174,1327174.0,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,2023.0,...,0.0,1.0,0.0,0.0,0.0,0.0,"[{'place_id': 307526888, 'licence': 'Data © Op...","{'place_id': 307526888, 'licence': 'Data © Ope...",13.2299741,52.4292052
293,22:13:00,2022-09-14 20:10:14.182771,2022,9,14,000010_1327172,1327172.0,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 236190135, 'licence': 'Data © Op...","{'place_id': 236190135, 'licence': 'Data © Ope...",13.5044167,52.49585


In [127]:
df5

,Time,Date,Year,Month,Day,unique_case_id,official_case_id,type_of_crime,location,year,...,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Output,Good_output,lon,lat
0,20:44:00,2023-05-30 20:10:14.182771,2023,5,30,000001_1318967,1318967.0,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121562969, 'licence': 'Data © Op...","{'place_id': 121562969, 'licence': 'Data © Ope...",13.29000220854009,52.50994745
1,11:38:00,2023-03-29 20:10:14.182771,2023,3,29,000003_1317835,1317835.0,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 103312933, 'licence': 'Data © Op...","{'place_id': 103312933, 'licence': 'Data © Ope...",13.1706652,52.5192835
2,00:47:00,2023-05-02 20:10:14.182771,2023,5,2,000004_1317465,1317465.0,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121847673, 'licence': 'Data © Op...","{'place_id': 121847673, 'licence': 'Data © Ope...",13.513130233356152,52.51865085
3,01:12:00,2022-11-27 20:10:14.182771,2022,11,27,000005_1317344,1317344.0,Property Damage,"Wedding, Togostraße",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 232223922, 'licence': 'Data © Op...","{'place_id': 232223922, 'licence': 'Data © Ope...",13.3375332,52.5548933
4,05:35:00,2022-07-06 20:10:14.182771,2022,7,6,000006_1317325,1317325.0,Attempted Suicide,Tempelhof-Schöneberg,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 307886045, 'licence': 'Data © Op...","{'place_id': 307886045, 'licence': 'Data © Ope...",13.388288723500661,52.4404422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,09:18:00,2022-10-08 20:10:14.182771,2022,10,8,000007_1327301,1327301.0,"Sexual Assault, Verbal Abuse",Lichtenberg,2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 308676399, 'licence': 'Data © Op...","{'place_id': 308676399, 'licence': 'Data © Ope...",7.4812268,48.9212959
291,07:24:00,2023-02-05 20:10:14.182771,2023,2,5,000008_1327284,1327284.0,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 113408058, 'licence': 'Data © Op...","{'place_id': 113408058, 'licence': 'Data © Ope...",13.364231910001136,52.54411815
292,14:19:00,2022-10-24 20:10:14.182771,2022,10,24,000009_1327174,1327174.0,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,2023.0,...,0.0,1.0,0.0,0.0,0.0,0.0,"[{'place_id': 307526888, 'licence': 'Data © Op...","{'place_id': 307526888, 'licence': 'Data © Ope...",13.2299741,52.4292052
293,22:13:00,2022-09-14 20:10:14.182771,2022,9,14,000010_1327172,1327172.0,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",2023.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 236190135, 'licence': 'Data © Op...","{'place_id': 236190135, 'licence': 'Data © Ope...",13.5044167,52.49585


In [128]:
df5.to_csv('dummy_table.csv')

In [133]:
df5.columns[17:35]

Index(['Hate Crime - Disability', 'Hate Crime - Gender',
       'Hate Crime - Gender Identity', 'Hate Crime - Religious',
       'Hate Crime - Sexual orientation', 'Hate Crime - Racial/Ethnicity',
       'Hate Crime - Ethnicity', 'Verbal Abuse/Verbal Assault',
       'Property Damage', 'Drug Offenses', 'General Assault', 'Sexual Assault',
       'Sexual Harassment', 'Property Crimes', 'Domestic Violence',
       'Missing Person', 'Traffic Incident', 'Unclassified'],
      dtype='object')

In [102]:
df5['Sexual Harassment'].value_counts()

Sexual Harassment
0.0    294
1.0      1
Name: count, dtype: int64

In [82]:
df5['Good_output'].value_counts()

Good_output
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          66
{'place_id': 307863134, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 164712, 'boundingbox': ['52.4704779', '52.5745093', '13.5168837', '13.6585034'], 'lat': '52.5225225', 'lon': '13.5876634', 'display_name': 'Marzahn-Hellersdorf, Berlin, Deutschland', 'class': 'boundary', 'type': 'administrative', 'importance': 0.6501948440349123, 'icon': 'https://nominatim.openstreetmap.org/ui/mapico

In [137]:
import random

def get_random_items(input_list, num_items):
    """
    Returns multiple items randomly from the given list.
    
    Args:
        input_list (list): The list from which to select items randomly.
        num_items (int): The number of random items to return.
        
    Returns:
        list: A list of randomly selected items.
    """
    return random.sample(input_list, num_items)


In [140]:
crime_types = [
    'Hate Crime - Disability', 'Hate Crime - Gender', 'Hate Crime - Gender Identity',
    'Hate Crime - Religious', 'Hate Crime - Sexual orientation',
    'Hate Crime - Racial/Ethnicity', 'Hate Crime - Ethnicity',
    'Verbal Abuse/Verbal Assault', 'Property Damage', 'Drug Offenses',
    'General Assault', 'Sexual Assault', 'Sexual Harassment',
    'Property Crimes', 'Domestic Violence', 'Missing Person',
    'Traffic Incident', 'Unclassified', 'I wanna see All'
]
random_items = get_random_items(crime_types, 3)


if len(random_items)>1:
    print(random_items)

['Verbal Abuse/Verbal Assault', 'Hate Crime - Religious', 'Property Damage']


In [143]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

# Sample data with columns for date, latitude, and longitude
data = pd.DataFrame({
    'Date': pd.date_range(start='2022-01-01', periods=100),
    'Latitude': np.random.uniform(-90, 90, 100),
    'Longitude': np.random.uniform(-180, 180, 100)
})

# Set up the options and their corresponding colors
option_colors = {
    'Option A': 'pink',
    'Option B': 'red',
    'Option C': 'green',
    'Option D': 'blue',
    'All Options': 'purple'
}

# Create a Streamlit selectbox for multiple options

# Filter the data based on the selected options
data['Option'].isin(option_colors)


KeyError: 'Option'

In [229]:
data['color'] = 'default color'

In [230]:
data

,Unnamed: 0,Time,Date,Year,Month,Day,unique_case_id,official_case_id,type_of_crime,location,...,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Output,Good_output,lon,lat,color
0,0,20:44:00,2023-05-30 20:10:14.182771,2023,5,30,000001_1318967,1318967.0,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121562969, 'licence': 'Data © Op...","{'place_id': 121562969, 'licence': 'Data © Ope...",13.290002,52.509947,default color
1,1,11:38:00,2023-03-29 20:10:14.182771,2023,3,29,000003_1317835,1317835.0,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 103312933, 'licence': 'Data © Op...","{'place_id': 103312933, 'licence': 'Data © Ope...",13.170665,52.519284,default color
2,2,00:47:00,2023-05-02 20:10:14.182771,2023,5,2,000004_1317465,1317465.0,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 121847673, 'licence': 'Data © Op...","{'place_id': 121847673, 'licence': 'Data © Ope...",13.513130,52.518651,default color
3,3,01:12:00,2022-11-27 20:10:14.182771,2022,11,27,000005_1317344,1317344.0,Property Damage,"Wedding, Togostraße",...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 232223922, 'licence': 'Data © Op...","{'place_id': 232223922, 'licence': 'Data © Ope...",13.337533,52.554893,default color
4,4,05:35:00,2022-07-06 20:10:14.182771,2022,7,6,000006_1317325,1317325.0,Attempted Suicide,Tempelhof-Schöneberg,...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 307886045, 'licence': 'Data © Op...","{'place_id': 307886045, 'licence': 'Data © Ope...",13.388289,52.440442,default color
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,290,09:18:00,2022-10-08 20:10:14.182771,2022,10,8,000007_1327301,1327301.0,"Sexual Assault, Verbal Abuse",Lichtenberg,...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 308676399, 'licence': 'Data © Op...","{'place_id': 308676399, 'licence': 'Data © Ope...",7.481227,48.921296,default color
225,291,07:24:00,2023-02-05 20:10:14.182771,2023,2,5,000008_1327284,1327284.0,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 113408058, 'licence': 'Data © Op...","{'place_id': 113408058, 'licence': 'Data © Ope...",13.364232,52.544118,default color
226,292,14:19:00,2022-10-24 20:10:14.182771,2022,10,24,000009_1327174,1327174.0,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,...,1.0,0.0,0.0,0.0,0.0,"[{'place_id': 307526888, 'licence': 'Data © Op...","{'place_id': 307526888, 'licence': 'Data © Ope...",13.229974,52.429205,default color
227,293,22:13:00,2022-09-14 20:10:14.182771,2022,9,14,000010_1327172,1327172.0,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",...,0.0,0.0,0.0,0.0,0.0,"[{'place_id': 236190135, 'licence': 'Data © Op...","{'place_id': 236190135, 'licence': 'Data © Ope...",13.504417,52.495850,default color
